# Pitch Detection Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

In [3]:
o = pd.read_csv('Duke Game 1 Center Field.csv')
o.head()

,Frame,cx,cy,width,height
0,179,0.360365,0.519166,0.019746,0.034254
1,180,0.362777,0.556992,0.020595,0.035851
2,181,0.366199,0.589505,0.018336,0.036508
3,182,0.366243,0.619478,0.017960,0.035255
4,183,0.366312,0.623461,0.017862,0.034723


In [17]:
cnt = o.groupby(['Frame']).count()

In [18]:
# Get information on the number of times when the ball is in the image
last_frame = 0
count = 0
for frame_num in cnt.index:
    
    if abs(last_frame - frame_num) != 1:
        count += 1
        
    last_frame = frame_num

print("Number of consecutive Streams with the ball:", count )

Number of consecutive Streams with the ball: 1234


## Get the Average spot of the ball

In [24]:
# Read a basic image from the game
img = cv2.imread('frames/frame00150.jpg')
# obtain the mean x,y,width, and height values
cx = o.mean()[1]
cy = o.mean()[2]
width = o.mean()[3]
height = o.mean()[4]
# image dimensions: (720, 1280)
image_width = len(img[0])
image_height = len(img)

# top left and right of the average rectangle
start = (int((cx - (width / 2))*image_width), int((cy - (height / 2))*image_height))
end = (int((cx + (width / 2))*image_width), int((cy + (height / 2))*image_height))
# draw avg rectangle over image and save it
img = cv2.rectangle(img, start, end, (255, 0, 255) , 2)
cv2.imwrite("Average_spot.png", img)

True